# **Implémentation d'un modèle de scoring**
## **Notebook 2/6 - Création de features & Assemblage des données**
*Sofia Chevrolat (Novembre 2020)*
___
Cette étude vise à réaliser un modèle permettant de prédire le risque de défaut de remboursement d'un prêt pour un client donné. Ce modèle doit être basé sur une variété de données (personnelles, en provenance de différentes sources bancaires, etc.).

Ce modèle est destiné à être servi via une API, elle-même appelée via un dashboard interactif. Le modèle devra donc être exporté une fois finalisé.
___
_**Remerciements**:<br>
Merci à mon compagnon [J. Duplan](https://www.linkedin.com/in/julian-duplan-64844a41/) pour les discussions intéressantes.<br>
Merci également à mon mentor [Samia Drappeau](https://www.linkedin.com/in/samiadrappeau) pour les échanges d'idées, les conseils et les encouragements !_
___


Ce notebook est organisé comme suit:

**0. Mise en place**
- 0.1 Chargement des librairies et fonctions utiles
- 0.2 Chargement et description du jeu de données
    
**1. Création de features & Assemblage des données**
- 1.1 Données APP_TRAIN
- 1.2 Données BUREAU
- 1.3 Données BUREAU_BALANCE
- 1.4 Données PREVIOUS APPLICATION
- 1.5 Données CASH
- 1.6 Données CARTE DE CRÉDIT
- 1.7 Données PAIEMENT

**2. Traitement des anomalies**
        
**3. Encodage des données**

**4. Export des données**

___
### 0. MISE EN PLACE

Dans cette première étape, le cadre de travail est mis en place, c'est-à-dire :
- Les librairies et packages Python nécessaires sont chargés
- Les fonctions utiles sont définies
- Le jeu de données est chargé
___

___
#### 0.1 CHARGEMENT DES LIBRAIRIES ET FONCTIONS UTILES

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import math
import pandas as pd
import numpy as np

In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [4]:
from sys import path
path.append("./Resources/functions")

import helper_functions as hf
import graphical_functions as gf

___
#### 0.2 CHARGEMENT ET DESCRIPTION DU JEU DE DONNÉES

In [5]:
app_train = pd.read_csv("./Resources/datasets/origin/application_train.csv")

bureau_balance = pd.read_csv("./Resources/datasets/origin/bureau_balance.csv")

bureau = pd.read_csv("./Resources/datasets/origin/bureau.csv")

credit = pd.read_csv("./Resources/datasets/origin/credit_card_balance.csv")

installments = pd.read_csv("./Resources/datasets/origin/installments_payments.csv")

cash = pd.read_csv("./Resources/datasets/origin/POS_CASH_balance.csv")

prev_app = pd.read_csv("./Resources/datasets/origin/previous_application.csv")

___
### 1. CRÉATION DE FEATURES & ASSEMBLAGE DES DONNÉES
___

___
#### 1.1 DONNÉES APP_TRAIN

Création d'une feature métier : 
-   Le ratio apport initial par rapport au prix du bien
-   Le ratio annuity / income 
-   Le ratio ancienneté emploi / âge

In [6]:
# Self financed on goods price ratio (%)
app_train["SELF_FINANCED_PERCENT"] = (app_train["AMT_GOODS_PRICE"] - app_train["AMT_CREDIT"])/app_train["AMT_GOODS_PRICE"]*100
app_train["SELF_FINANCED_PERCENT"] = app_train["SELF_FINANCED_PERCENT"].map(lambda x: 0 if x<0 else x)

In [7]:
# ANNUITY ON INCOME ratio (%)
app_train["ANNUITY_ON_INCOME"] = app_train["AMT_ANNUITY"] / app_train["AMT_INCOME_TOTAL"] * 100

In [8]:
# Days employed on age ratio (%)
app_train['DAYS_EMPLOYED_PERCENT'] = app_train['DAYS_EMPLOYED'] / app_train['DAYS_BIRTH']

___
#### 1.2 DONNÉES BUREAU

Création des features suivantes :
-   Nombre de prêts bancaires précédents

Création des features suivantes par prêt par client :
-   Pour chaque feature qualitative :
    - nombre
    - moyenne
    - somme
- Pour chaque feature quantitative : 
    - nombre
    - moyenne
    - maximum
    - minimum
    - somme

In [9]:
# Convert types for less memory usage
bureau = hf.convert_types(bureau, print_info=True)

Original Memory Usage: 0.23 gb.
New Memory Usage: 0.1 gb.


In [10]:
# Creation de previous_loan_counts
previous_loan_counts = bureau.groupby('SK_ID_CURR', 
                                      as_index=False)['SK_ID_BUREAU']\
                             .count()\
                             .rename(columns = {'SK_ID_BUREAU':\
                                                'previous_loan_counts'})

# Merge de previous_loan_counts dans train on SK_ID_CURR, left
app_train = app_train.merge(previous_loan_counts, 
                            on = 'SK_ID_CURR', how = 'left')

# fillna(0) dans train
app_train['previous_loan_counts'] = app_train['previous_loan_counts']\
                                    .fillna(0)

In [11]:
# Creation of bureau_counts containing for each possible value 
# of the qualitative features 2 new feature : count and 
# normalized count

bureau_counts = hf.agg_categorical(bureau, 
                                     group_var = 'SK_ID_CURR', 
                                     df_name = 'bureau')

In [12]:
# Creation of bureau_agg containing for each possible value
# of the quantitative features 5 new features : count, max,
# mean, min and sum
 
bureau_agg = hf.agg_numeric(bureau.drop(columns = ['SK_ID_BUREAU']), 
                            group_var = 'SK_ID_CURR', 
                            df_name = 'bureau')

In [13]:
# Insert computed features into training data

# Merge bureau_counts dans app_train
app_train = app_train.merge(bureau_counts, on="SK_ID_CURR", how="left")

# Merge bureau_agg dans app_train
app_train = app_train.merge(bureau_agg, on="SK_ID_CURR", how="left")

# Suppression des colonnes missing
app_train = hf.remove_missing_columns(app_train)

There are 0 columns with greater than 90% missing values.


___
#### 1.3 DONNÉES BUREAU_BALANCE

Création des features suivantes par prêt par client :
-   Pour chaque features qualitative :
    - nombre
    - moyenne
    - somme
-   Features mathématiques pour chaque features quantitative par prêt par client : 
    - nombre
    - moyenne
    - maximum
    - minimum
    - somme

In [14]:
# Convert types for less memory usage
bureau_balance = hf.convert_types(bureau_balance, print_info=True)

Original Memory Usage: 0.66 gb.
New Memory Usage: 0.25 gb.


In [15]:
# Comptage des catégories
# Counts of each type of status for each previous loan
bureau_balance_counts = hf.agg_categorical(bureau_balance, 
                                           group_var = 'SK_ID_BUREAU', 
                                           df_name = 'bureau_balance')

# Creation de bureau_balance_agg 
# Calculate value count statistics for each `SK_ID_CURR` 
bureau_balance_agg = hf.agg_numeric(bureau_balance, 
                                    group_var = 'SK_ID_BUREAU', 
                                    df_name = 'bureau_balance')

# Creation de bureau_by_loan
# Dataframe grouped by the loan
bureau_by_loan = bureau_balance_agg.merge(bureau_balance_counts, 
                                          right_index = True, 
                                          left_on = 'SK_ID_BUREAU', 
                                          how = 'outer')

# Merge to include the SK_ID_CURR - Possibly several rows per client
bureau_by_loan = bureau_by_loan.merge(bureau[['SK_ID_BUREAU', 'SK_ID_CURR']], 
                                      on = 'SK_ID_BUREAU', 
                                      how = 'left')

# Creation de bureau_balance_by_client - One row per client
bureau_balance_by_client = hf.agg_numeric(bureau_by_loan.drop(columns = ['SK_ID_BUREAU']), 
                                          group_var = 'SK_ID_CURR', 
                                          df_name = 'client')

In [16]:
# Merge bureau_balance_by_client dans app_train
app_train = app_train.merge(bureau_balance_by_client, on="SK_ID_CURR", how="left")

# Suppression des colonnes missing
app_train = hf.remove_missing_columns(app_train)

There are 0 columns with greater than 90% missing values.


___
#### 1.4 DONNÉES PREVIOUS APPLICATION

Création des features suivantes :
- Features mathématiques pour chaque feature qualitative par client : 
    - somme
    - nombre
    - moyenne
- Features mathématiques pour chaque features quantitative par client : 
    - nombre
    - moyenne
    - maximum
    - minimum
    - somme

In [17]:
# Convert types de previous
prev_app = hf.convert_types(prev_app, print_info=True)

Original Memory Usage: 0.49 gb.
New Memory Usage: 0.16 gb.


In [18]:
# Creation de previous_agg
prev_agg = hf.agg_numeric(prev_app, 'SK_ID_CURR', 'previous')

# Creation de previous_counts
prev_counts = hf.agg_categorical(prev_app, 'SK_ID_CURR', 'previous')

In [19]:
# Merge previous_counts dans app_train
app_train = app_train.merge(prev_counts, on="SK_ID_CURR", how="left")

# Merge previous_agg dans app_train
app_train = app_train.merge(prev_agg, on="SK_ID_CURR", how="left")

# Suppression des colonnes missing
app_train = hf.remove_missing_columns(app_train)

There are 6 columns with greater than 90% missing values.


___
#### 1.5 DONNÉES CASH

Création des features suivantes :
- Features mathématiques pour chaque feature qualitative par client : 
    - somme
    - nombre
    - moyenne
- Features mathématiques pour chaque features quantitative par client : 
    - nombre
    - moyenne
    - maximum
    - minimum
    - somme

In [20]:
# Convert types de cash
cash = hf.convert_types(cash, print_info=True)

Original Memory Usage: 0.64 gb.
New Memory Usage: 0.29 gb.


In [21]:
# Creation de cash_by_client
cash_by_client = hf.aggregate_client(cash, 
                                     group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], 
                                     df_names = ['cash', 'client'])

# Merge cash_by_client dans app_train
app_train = app_train.merge(cash_by_client, on = 'SK_ID_CURR', how = 'left')

# Suppression des colonnes missing
app_train = hf.remove_missing_columns(app_train)

There are 0 columns with greater than 90% missing values.


___
#### 1.6 DONNÉES CARTE DE CRÉDIT

Création des features suivantes :
- Features mathématiques pour chaque feature qualitative par client : 
    - somme
    - nombre
    - moyenne
- Features mathématiques pour chaque features quantitative par client : 
    - nombre
    - moyenne
    - maximum
    - minimum
    - somme

In [22]:
# Convert types de credit
credit = hf.convert_types(credit, print_info=True)

Original Memory Usage: 0.71 gb.
New Memory Usage: 0.34 gb.


In [23]:
# Creation de credit_by_client
credit_by_client = hf.aggregate_client(credit, 
                                       group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], 
                                       df_names = ['credit', 'client'])

# Merge credit_by_client dans app_train
app_train = app_train.merge(credit_by_client, on="SK_ID_CURR", how="left")

# Suppression des colonnes missing
app_train = hf.remove_missing_columns(app_train)

There are 0 columns with greater than 90% missing values.


___
#### 1.7 DONNÉES PAIEMENT

Création des features suivantes :
- Features mathématiques pour chaque feature qualitative par client : 
    - somme
    - nombre
    - moyenne
- Features mathématiques pour chaque features quantitative par client : 
    - nombre
    - moyenne
    - maximum
    - minimum
    - somme

In [24]:
# Convert types de installments
installments = hf.convert_types(installments, print_info = True)

Original Memory Usage: 0.87 gb.
New Memory Usage: 0.44 gb.


In [25]:
# Creation de installments_by_clients
installments_by_client = hf.aggregate_client(installments, 
                                             group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], 
                                             df_names = ['installments', 'client'])

# Merge installments_by_clietns dans app_train
app_train = app_train.merge(installments_by_client, on = 'SK_ID_CURR', how = 'left')

# Suppression des colonnes missing
app_train = hf.remove_missing_columns(app_train)

There are 0 columns with greater than 90% missing values.


___
### 2. TRAITEMENT DES ANOMALIES

L'analyse exploratoire a fait ressortir une anomalie identique pour 18% des clients : une ancienneté au travail de 365000 jours. 

Nous allons remplacer cette valeur anormale par NaN, tout en conservant l'information que ces clients avaient cette anomalie.
___

In [26]:
# Create an anomalous flag column
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243

# Replace the anomalous values with nan
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

___
### 3. ENCODAGE DES DONNÉES

Transformation des variables qualitatives en variables quantitatives via l'utilisation de :
- label encoding pour les variables qualitatives comptant moins de 2 modalités
- one hot encoding pour les autres variables qualitatives
___

In [27]:
# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in app_train:
    if app_train[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(app_train[col].unique())) <= 2:
            # Train on the data
            le.fit(app_train[col])
            # Transform data
            app_train[col] = le.transform(app_train[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

3 columns were label encoded.


In [28]:
# one-hot encoding of categorical variables
app_train = pd.get_dummies(app_train)

___
### 4. EXPORT DES DONNÉES
___

In [29]:
app_train.to_csv("./Resources/datasets/assembled/full_training_data.csv", index=False, chunksize=500)